This notebook operates on the Microsoft Planetary Computer. You need to make an API token connection or run it in the MPC Hub as it operates on STAC cataloged data stored in Azure where the MPC Pangeo environment is running. It also works against a cache of the EPA Facilities Registry Service data that I stored in my home account there, transformed from zipped CSV to Parquet. You could pull the FRS data in a different way.

I'm exploring how we build in confirmed linkages to what's already in the FRS data - facilities to tribal, state, county, and city (in some cases) - and also how we build the linking process to congressional districts, hydrographic units, ecosystem classifications, and other "divisions" of interest. I'm also working on getting to more detail about the facilities such as the companies who own and/or operate them.

To do this, we need to understand the level of accuracy in the data, improve accuracy where we can do so reasonably and confidently, and record uncertainty in a way that informs further analyses with these data and helps EPA or others improve the underlying systems. FRS is a big dataset with over 2.7M records. Many of those are old records for facilities that are no longer active. We will not and do not need to achieve total accuracy, but if we do need a solid foundation from which to build and an understanding of where the inaccuracies lie.

In this notebook, I'm leveraging the cache of [2020 U.S. Census data](https://planetarycomputer.microsoft.com/dataset/us-census) the MPC has spun up as public data. Full provenance is not documented, but I'm reasonably confident in the basics of tribal areas, states, counties, and congressional districts. Cities get a little fuzzy as this is based on Census areas that are not semantically aligned with FRS. Making this linkage here with actual 2020 Census demographic data may also prove useful if we want to bring any of that into analytical work.

In [2]:
import geopandas as gpd
import dask_geopandas
import contextily as ctx
import seaborn as sns
import planetary_computer
import pystac_client
import pandas as pd

from pathlib import Path
from glob import glob


In [3]:
data_catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

search = data_catalog.search(collections=["us-census"])
items = {item.id: item for item in search.items()}
list(items)

['2020-census-blocks-population',
 '2020-census-blocks-geo',
 '2020-cb_2020_us_vtd_500k',
 '2020-cb_2020_us_unsd_500k',
 '2020-cb_2020_us_ttract_500k',
 '2020-cb_2020_us_tract_500k',
 '2020-cb_2020_us_tbg_500k',
 '2020-cb_2020_us_state_500k',
 '2020-cb_2020_us_sldu_500k',
 '2020-cb_2020_us_sldl_500k',
 '2020-cb_2020_us_scsd_500k',
 '2020-cb_2020_us_region_500k',
 '2020-cb_2020_us_place_500k',
 '2020-cb_2020_us_nectadiv_500k',
 '2020-cb_2020_us_necta_500k',
 '2020-cb_2020_us_nation_5m',
 '2020-cb_2020_us_metdiv_500k',
 '2020-cb_2020_us_elsd_500k',
 '2020-cb_2020_us_division_500k',
 '2020-cb_2020_us_csa_500k',
 '2020-cb_2020_us_cousub_500k',
 '2020-cb_2020_us_county_within_cd116_500k',
 '2020-cb_2020_us_county_500k',
 '2020-cb_2020_us_concity_500k',
 '2020-cb_2020_us_cnecta_500k',
 '2020-cb_2020_us_cd116_500k',
 '2020-cb_2020_us_cbsa_500k',
 '2020-cb_2020_us_bg_500k',
 '2020-cb_2020_us_aitsn_500k',
 '2020-cb_2020_us_aiannh_500k',
 '2020-cb_2020_72_subbarrio_500k',
 '2020-cb_2020_02_anrc_

In [16]:
# Get tribal, state, county, and congressional district datasets
item_aiannh = items["2020-cb_2020_us_aiannh_500k"]
asset_aiannh = item_aiannh.assets["data"]
gdf_aiannh = gpd.read_parquet(
    asset_aiannh.href,
    storage_options=asset_aiannh.extra_fields["table:storage_options"]
)
gdf_aiannh.to_crs(epsg=4326, inplace=True)

item_states = items["2020-cb_2020_us_state_500k"]
asset_states = item_states.assets["data"]
gdf_states = gpd.read_parquet(
    asset_states.href,
    storage_options=asset_states.extra_fields["table:storage_options"]
)
gdf_states.to_crs(epsg=4326, inplace=True)

item_counties = items['2020-cb_2020_us_county_500k']
asset_counties = item_counties.assets['data']
gdf_counties = gpd.read_parquet(
    asset_counties.href,
    storage_options=asset_counties.extra_fields["table:storage_options"]
)
gdf_counties.to_crs(epsg=4326, inplace=True)

item_cong_dist = items["2020-cb_2020_us_cd116_500k"]
asset_cong_dist = item_cong_dist.assets["data"]
gdf_cong_dist = gpd.read_parquet(
    asset_cong_dist.href,
    storage_options=asset_cong_dist.extra_fields["table:storage_options"]
)
gdf_cong_dist.to_crs(epsg=4326, inplace=True)

# Facilities Registry Service

This is the part of the USEPA ECHO system that is at the heart of the regulatory process. Facilities regulated under the Clean Water Act, Clean Air Act, and other legal vehicles based on the type of industry and work the facilities do have to register through what is now an online process. The FRS dataset has both current and legacy information, and it's noisy in a number of ways. I'm exploring how to identify and flag the messiness as I bring these records into the Wikibase knowledge graph I'm developing.

FRS and other ECHO data are available via bulk download and an API. There is misalignment between the two that I've found, which is its own issue. But the best way to deal with analyzing through the data is through bulk access. For FRS data available [here](https://echo.epa.gov/tools/data-downloads), I have another process that unzips them as CSV from source and caches them in my user directory on the MPC so I can operate everything there and don't have to retrieve the data ever time.

In [6]:
# Pull cached FRS facilities
# Pre-cached in my MPC Hub folder space
frs_facilities = pd.read_parquet('data/FRS_FACILITIES.parquet')

## With and without coordinates

Some FRS records have point coordinates associated with them. We'd like to get accurate coordinates for every facility we want to examine in some way as this gives us the best way to link them to anything we can link spatially such as political and ecological boundaries/divisions of various kinds. Just a quick look, though, shows that there are "collisions" in the data where what a coordinate points to is not where other attributes (address, city, state, county) indicate the facility is located. We have to figure out what to do with these cases. In other cases, there are point coordinates but some of the other information is missing, so the coordinates could be used to fill in the blanks.

These are all good cases where the Wikibase approach I'm following can help as it allows us to maintain multiple assertions (claims) about the same characteristic at the same time. We can record that the raw FRS data says one thing while our algorithm that combines several kinds of reasoning says something else. We can then decide which of those two assertions we want to trust for a particular purpose and point to exactly what we based that decision upon.

Here, I'm teasing out those registrants with coordinates and making sure they are actionable float values we can pull into a GeoDataFrame.

In [7]:
# Separate FRS records with and without coordinates
frs_with_coords = frs_facilities[
    frs_facilities.LATITUDE_MEASURE.notnull()
    &
    frs_facilities.LONGITUDE_MEASURE.notnull()
].reset_index(drop=True)

frs_with_coords['lat'] = frs_with_coords.LATITUDE_MEASURE.apply(float)
frs_with_coords['lon'] = frs_with_coords.LONGITUDE_MEASURE.apply(float)

frs_with_coords.drop(columns=['LATITUDE_MEASURE','LONGITUDE_MEASURE'], inplace=True)

In [9]:
# Create GeoDataFrame for FRS with coordinates
gdf_frs_with_coords = gpd.GeoDataFrame(
    frs_with_coords,
    geometry=gpd.points_from_xy(
        frs_with_coords.lon, 
        frs_with_coords.lat
    ),
    crs='EPSG:4326'
)
print(len(gdf_frs_with_coords))

display(gdf_frs_with_coords.head())

1955805


,FAC_NAME,FAC_STREET,FAC_CITY,FAC_STATE,FAC_ZIP,REGISTRY_ID,FAC_COUNTY,FAC_EPA_REGION,lat,lon,geometry
0,WABASH NATIONAL WOOD PRODUCTS,339 INDUSTRIAL PARK RD,HARRISON,AR,72601,110001710195,BOONE,06,36.249327,-93.149697,POINT (-93.14970 36.24933)
1,WESTROCK CONVERTING LLC - CONWAY,198 COMMERCE STREET,CONWAY,AR,72032,110001710220,FAULKNER,06,35.154107,-92.747633,POINT (-92.74763 35.15411)
2,MERIDIAN AGGREGATES COMPANY-BLK RCK QURY,133 LAWRENCE 208,BLACK ROCK,AR,72415,110001710239,LAWRENCE,06,36.156856,-91.148467,POINT (-91.14847 36.15686)
3,AMFUEL PLANT II,1706 NORTH VINE,MAGNOLIA,AR,71753,110001710248,COLUMBIA,06,33.282994,-93.245476,POINT (-93.24548 33.28299)
4,HUGHES MISSILE SYSTEMS CO,204 QUACHITA 212,EAST CAMDEN,AR,71701,110001710257,OUACHITA,06,33.619670,-92.759213,POINT (-92.75921 33.61967)


## Points in States and Counties

States and counties are two geographic location identifiers in the basic FRS source. According to documentation, FAC_STATE should be the two-character State abbreviation and FAC_COUNTY should be some form of the county name, presumably within an indicated state. There are alreaady some issues with state identifiers that aren't U.S. States, Counties that aren't in the State for the same record, and other presumed mistakes. Some of this we can identify and examine just by looking at properties in the data.

With boundaries for U.S. states and counties, we can start to check the alignment (or not) with point coordinates when those are supplied. We can do that with a simple join. Because these aren't massive, we won't stress out the 32GB of available memory on the small MPC machine we are using, and we don't really need to worry about parallelizing this with dask_geopandas just yet. We'll get to that at some point, though.

If we could confirm a good point coordinate, then we can derive everything else from there. One approach is to identify something like the number of confirming details. If a record has a point location and that location aligns with the indicated state, that's one level of conformance. If the point is also in the indicated county, and we can connect the dots on the naming "convention," that's another level. We might go further and hit metropolitan areas and add another level of confidence.

The initial spatial joins are just giving us FRS records and their corresponding State/County records where the FRS points are located within either a state or a county. From there, we can look to see how other attributes align.

In [17]:
frs_points_in_states = gpd.sjoin(
    gdf_frs_with_coords, 
    gdf_states, 
    how='inner', 
    predicate='within'
)

frs_points_in_counties = gpd.sjoin(
    gdf_frs_with_coords, 
    gdf_counties, 
    how='inner', 
    predicate='within'
)

In [11]:
print("How many registrants have coordinates located in at least some state?", len(frs_points_in_states))
print("How many registrants have point coordinates not located in any U.S. state?", len(gdf_frs_with_coords[~gdf_frs_with_coords.REGISTRY_ID.isin(frs_points_in_states.REGISTRY_ID)]))

How many registrants have coordinates located in at least some state? 1950249
How many registrants have point coordinates not located in any U.S. state? 5556


In [12]:
frs_points_in_states[frs_points_in_states.FAC_STATE == frs_points_in_states.STUSPS]

,FAC_NAME,FAC_STREET,FAC_CITY,FAC_STATE,FAC_ZIP,REGISTRY_ID,FAC_COUNTY,FAC_EPA_REGION,lat,lon,...,index_right,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER
0,WABASH NATIONAL WOOD PRODUCTS,339 INDUSTRIAL PARK RD,HARRISON,AR,72601,110001710195,BOONE,06,36.249327,-93.149697,...,53,05,68085,0400000US05,05,AR,Arkansas,00,134660850501,3121867339
1,WESTROCK CONVERTING LLC - CONWAY,198 COMMERCE STREET,CONWAY,AR,72032,110001710220,FAULKNER,06,35.154107,-92.747633,...,53,05,68085,0400000US05,05,AR,Arkansas,00,134660850501,3121867339
2,MERIDIAN AGGREGATES COMPANY-BLK RCK QURY,133 LAWRENCE 208,BLACK ROCK,AR,72415,110001710239,LAWRENCE,06,36.156856,-91.148467,...,53,05,68085,0400000US05,05,AR,Arkansas,00,134660850501,3121867339
3,AMFUEL PLANT II,1706 NORTH VINE,MAGNOLIA,AR,71753,110001710248,COLUMBIA,06,33.282994,-93.245476,...,53,05,68085,0400000US05,05,AR,Arkansas,00,134660850501,3121867339
4,HUGHES MISSILE SYSTEMS CO,204 QUACHITA 212,EAST CAMDEN,AR,71701,110001710257,OUACHITA,06,33.619670,-92.759213,...,53,05,68085,0400000US05,05,AR,Arkansas,00,134660850501,3121867339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1938829,"INNOVATIVE ASSET GROUP, LLC PROJECT",REM. PLOT NO. 50 ESTATE ORANGE GROVE,ST. CROIX,VI,00820,110071335634,None,02,17.744125,-64.719027,...,5,78,1802710,0400000US78,78,VI,United States Virgin Islands,00,348021914,1550236187
1947161,FLAMBOYAN ON THE BAY RESORT & VILLAS,6200 MAGENS BAY RD.,ST. THOMAS,VI,00802,110071362340,None,02,18.359172,-64.915405,...,5,78,1802710,0400000US78,78,VI,United States Virgin Islands,00,348021914,1550236187
1952079,HIGH QUALITY CONCRETE,NO. 8 ESTATE COTTAGE,ST. CROIX,VI,00820,110071358945,None,02,17.722908,-64.755623,...,5,78,1802710,0400000US78,78,VI,United States Virgin Islands,00,348021914,1550236187
1952306,CHITOLIE TRUCKING SERVICE,NO. 2&4 CASAVA GARDENS,ST. CROIX,VI,00851,110071359042,None,02,17.719528,-64.742980,...,5,78,1802710,0400000US78,78,VI,United States Virgin Islands,00,348021914,1550236187


In [19]:
frs_points_in_counties[frs_points_in_counties.FAC_COUNTY.str.lower() == frs_points_in_counties.NAME.str.lower()]

,FAC_NAME,FAC_STREET,FAC_CITY,FAC_STATE,FAC_ZIP,REGISTRY_ID,FAC_COUNTY,FAC_EPA_REGION,lat,lon,...,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER
0,WABASH NATIONAL WOOD PRODUCTS,339 INDUSTRIAL PARK RD,HARRISON,AR,72601,110001710195,BOONE,06,36.249327,-93.149697,...,00069897,0500000US05009,05009,Boone,Boone County,AR,Arkansas,06,1527375340,31797989
330,OZARK AUTOMOTIVE DISTRIBUTORS,1524 N MAIN,HARRISON,AR,72601,110003397057,BOONE,06,36.248065,-93.118875,...,00069897,0500000US05009,05009,Boone,Boone County,AR,Arkansas,06,1527375340,31797989
1025,HARRISON DAILY TIMES,111 W RUSH ST,HARRISON,AR,72601,110003405039,BOONE,06,36.230878,-93.108002,...,00069897,0500000US05009,05009,Boone,Boone County,AR,Arkansas,06,1527375340,31797989
1286,US ARMY RESERVE CENTER,601 W SHERMAN ST,HARRISON,AR,72601,110003392891,BOONE,06,36.241050,-93.113500,...,00069897,0500000US05009,05009,Boone,Boone County,AR,Arkansas,06,1527375340,31797989
1292,RAYS SERVICE,524 VINE ST,HARRISON,AR,72601,110003393024,BOONE,06,36.225207,-93.107496,...,00069897,0500000US05009,05009,Boone,Boone County,AR,Arkansas,06,1527375340,31797989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1509966,BRIDGE INVESTMENT GROUP TINIAN COMMERCIAL TERM...,TINIAN SEAPORT,TINIAN,MP,96952,110070111888,TINIAN,09,14.965000,145.621800,...,01805248,0500000US69120,69120,Tinian,Tinian Municipality,MP,Commonwealth of the Northern Mariana Islands,12,108237606,632893009
1810751,BHOJWANI LAND CLEARING (TINIAN),PO BOX 520937,TINIAN,MP,96952,110071177100,TINIAN,09,14.993700,145.652400,...,01805248,0500000US69120,69120,Tinian,Tinian Municipality,MP,Commonwealth of the Northern Mariana Islands,12,108237606,632893009
1833932,TINIAN SEABEE EXPEDITIONARY CAMP,8TH AVENUE,TINIAN,MP,96952,110070693776,TINIAN,09,15.044300,145.611500,...,01805248,0500000US69120,69120,Tinian,Tinian Municipality,MP,Commonwealth of the Northern Mariana Islands,12,108237606,632893009
797309,"MOBIL BULK PLANT ROTA, CNMI",SONGSONG,ROTA,MP,96951,110043671363,ROTA,09,14.139730,145.144929,...,01805246,0500000US69100,69100,Rota,Rota Municipality,MP,Commonwealth of the Northern Mariana Islands,12,85096798,539092797


These first two basic indicators might narrow down more of our problematic records. We have a reasonable indication of records that have what's probably valid values for state and county along with coordinates that put the facility record in the indicated division. For the knowledge graph, we will link these explicitly via claims to those state and county entities. Since those three pieces of information are in concurrence with one another, we can probably build on facility point location to get relationship to linkages not explicitly in the data - tribal areas and congressional districts.

In [20]:
state_confirmed_registry_ids = frs_points_in_states[frs_points_in_states.FAC_STATE == frs_points_in_states.STUSPS].REGISTRY_ID
county_confirmed_registry_ids = frs_points_in_counties[frs_points_in_counties.FAC_COUNTY.str.lower() == frs_points_in_counties.NAME.str.lower()].REGISTRY_ID

In [24]:
frs_points_in_aiannh = gpd.sjoin(
    gdf_frs_with_coords[
        (gdf_frs_with_coords.REGISTRY_ID.isin(state_confirmed_registry_ids))
        |
        (gdf_frs_with_coords.REGISTRY_ID.isin(county_confirmed_registry_ids))
    ], 
    gdf_aiannh, 
    how='inner', 
    predicate='within'
)

In [25]:
frs_points_in_aiannh

,FAC_NAME,FAC_STREET,FAC_CITY,FAC_STATE,FAC_ZIP,REGISTRY_ID,FAC_COUNTY,FAC_EPA_REGION,lat,lon,...,index_right,AIANNHCE,AIANNHNS,AFFGEOID,GEOID,NAME,NAMELSAD,LSAD,ALAND,AWATER
46,NYE FRONTIER FORD,2701 E MT VILLAGE DR,WASILLA,AK,99654,110003041440,MATANUSKA-SUSITNA,10,61.573550,-149.381363,...,291,6785,2418661,2500000US6785,6785,Knik,Knik ANVSA,79,18093901349,978064236
702,BEAR AIR,3151 E PALMDALE DR,WASILLA,AK,99654,110003039694,MATANUSKA-SUSITNA,10,61.595195,-149.369716,...,291,6785,2418661,2500000US6785,6785,Knik,Knik ANVSA,79,18093901349,978064236
2020,CHEVRON USA INC WASILLA SS 92291,601 W PARKS HWY,WASILLA,AK,99654,110003039195,MATANUSKA-SUSITNA,10,61.582030,-149.466510,...,291,6785,2418661,2500000US6785,6785,Knik,Knik ANVSA,79,18093901349,978064236
2648,USEPA WILLOW MERCURY SPILL,"S PARKS HWY, MI 77",TRAPPER CREEK,AK,99683,110007918584,MATANUSKA-SUSITNA,10,62.277886,-150.246577,...,291,6785,2418661,2500000US6785,6785,Knik,Knik ANVSA,79,18093901349,978064236
2655,GOLDORADO GOLD & ELDORADO CREEK MINE SITE,"TALKEETNA MTNS C-6, D-6",CANTWELL,AK,99729,110009004175,DENALI,10,62.748720,-149.631000,...,291,6785,2418661,2500000US6785,6785,Knik,Knik ANVSA,79,18093901349,978064236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1874215,AKELA FLATS WASTE WATER TREATMENT PLANT,EXIT 102 OFF OF I10,AKELA FLATS,NM,88030,110071254273,None,06,32.241232,-107.422789,...,45,1270,2711352,2500000US1270,1270,Fort Sill Apache,Fort Sill Apache Indian Reservation,96,121466,0
1922734,LVWD SANTIAGO BUSTAMANTE SANITARY SEWER,SANTIAGO BUSTAMANTE AND JOSE GRANILLO,EL PASO,TX,79927,110071335318,None,06,31.618100,-106.291100,...,662,4755,2419529,2500000US4755,4755,Ysleta del Sur,Ysleta del Sur Pueblo and Off-Reservation Trus...,84,13058989,0
1923122,LVWD SANTIAGO BUSTAMANTE SANITARY SEWER,1557 FM ROAD 1110,CLINT,TX,79836,110071337352,None,06,31.618100,-106.291100,...,662,4755,2419529,2500000US4755,4755,Ysleta del Sur,Ysleta del Sur Pueblo and Off-Reservation Trus...,84,13058989,0
1933550,HILL TOP SUBDIVISION_2,MAJOR ROAD,HAINES,AK,99827,110071238604,HAINES,10,59.226090,-135.451060,...,334,6315,2418826,2500000US6315,6315,Chilkoot,Chilkoot ANVSA,79,1223953,0


In [26]:
frs_points_in_cong_dist = gpd.sjoin(
    gdf_frs_with_coords[
        (gdf_frs_with_coords.REGISTRY_ID.isin(state_confirmed_registry_ids))
        |
        (gdf_frs_with_coords.REGISTRY_ID.isin(county_confirmed_registry_ids))
    ], 
    gdf_cong_dist, 
    how='inner', 
    predicate='within'
)

In [27]:
frs_points_in_cong_dist

,FAC_NAME,FAC_STREET,FAC_CITY,FAC_STATE,FAC_ZIP,REGISTRY_ID,FAC_COUNTY,FAC_EPA_REGION,lat,lon,...,index_right,STATEFP,CD116FP,AFFGEOID,GEOID,NAMELSAD,LSAD,CDSESSN,ALAND,AWATER
0,WABASH NATIONAL WOOD PRODUCTS,339 INDUSTRIAL PARK RD,HARRISON,AR,72601,110001710195,BOONE,06,36.249327,-93.149697,...,51,05,3,5001600US0503,0503,Congressional District 3,C2,116,13985796400,374570670
8,LABEL ART INC,5721 ZERO ST,FORT SMITH,AR,72903,110003399992,SEBASTIAN,06,35.327000,-94.372600,...,51,05,3,5001600US0503,0503,Congressional District 3,C2,116,13985796400,374570670
12,DUCOMMUN LABARGE ELECTRONICS,810 CHAMPLIN RD,BERRYVILLE,AR,72616,110003400052,CARROLL,06,36.378990,-93.580220,...,51,05,3,5001600US0503,0503,Congressional District 3,C2,116,13985796400,374570670
15,SCHLUMBERGER WELL SERVICE,1011 SOUTH ZERO STREET,FORT SMITH,AR,72901,110003400098,SEBASTIAN,06,35.328212,-94.424382,...,51,05,3,5001600US0503,0503,Congressional District 3,C2,116,13985796400,374570670
17,JOHNSON SHOP,6603 MCGUIRE ST,FAYETTEVILLE,AR,72704,110003400123,WASHINGTON,06,36.123410,-94.176040,...,51,05,3,5001600US0503,0503,Congressional District 3,C2,116,13985796400,374570670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1954938,SCHNEIDER ELECTRIC EL PASO PLANT 5,1600 NORTHWESTERN DR,EL PASO,TX,79912,110071369989,None,06,31.891930,-106.578110,...,438,48,16,5001600US4816,4816,Congressional District 16,C2,116,1841189026,3570589
1955120,PASEO DEL NORTE SHELL,7721 PASEO DEL NORTE,EL PASO,TX,79912,110071371316,None,06,31.885400,-106.573200,...,438,48,16,5001600US4816,4816,Congressional District 16,C2,116,1841189026,3570589
1955201,ENCHANTED HILLS UNIT FIVE,SITE LOCATED AT THE CORNER OF NORTHWESTERN DR....,EL PASO,TX,79912,110071375300,None,06,31.921870,-106.573030,...,438,48,16,5001600US4816,4816,Congressional District 16,C2,116,1841189026,3570589
1955381,LUCKETT WASTEWATER REPLACEMENT,PINAR DEL RIO DR. & WEST VALLEY CIRCLE,EL PASO,TX,79902,110071377957,None,06,31.870213,-106.593660,...,438,48,16,5001600US4816,4816,Congressional District 16,C2,116,1841189026,3570589
